In [ ]:
import os
import shutil


%cd /kaggle/working


repo_name = "benchmarking-generative-models-for-domain-adaptation"
if os.path.exists(repo_name):
    shutil.rmtree(repo_name)


print("Clono la repository...")
!git clone --recursive https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation.git
%cd {repo_name}


In [ ]:
%cd /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet

os.makedirs('initmodel', exist_ok=True)


source_path = "/kaggle/input/datasets/antoniorosano/weights-resnet/resnet50_v2.pth" 
destination_path = "./initmodel/resnet50_v2.pth" 


if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print(f"Pesi copiati con successo in: {destination_path}")
else:
    print(f"Errore: Non trovo il file in {source_path}")
    print("File disponibili in input:")
    for root, dirs, files in os.walk("/kaggle/input"):
        for file in files:
            print(os.path.join(root, file))

In [ ]:
import os

print("Applicazione Patch al codice...")


train_py = "tool/train.py"
if os.path.exists(train_py):
    with open(train_py, 'r') as f: txt = f.read()
    if "checkpoint['state_dict']" in txt and "strict=False" not in txt:
        txt = txt.replace("model.load_state_dict(checkpoint['state_dict'])", 
                          "model.load_state_dict(checkpoint['state_dict'], strict=False)")
        with open(train_py, 'w') as f: f.write(txt)
        print("Patch 'strict=False' applicata a train.py")


dataset_code = """
import os
import os.path
import cv2
import numpy as np
from torch.utils.data import Dataset

def make_dataset(split='train', data_root=None, data_list=None):
    image_label_list = []
    list_read = open(data_list).readlines()
    for line in list_read:
        line = line.strip().split(' ')
        image_label_list.append((line[0], line[1])) 
    return image_label_list

class SemData(Dataset):
    def __init__(self, split='train', data_root=None, data_list=None, transform=None):
        self.split = split
        self.data_list = make_dataset(split, data_root, data_list)
        self.transform = transform
    def __len__(self): return len(self.data_list)
    def __getitem__(self, index):
        image_path, label_path = self.data_list[index]
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.float32(image)
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        if label.ndim == 3: label = label[:, :, 0]
        # Sanitizzazione: tutto ciò che è >= 25 diventa ignore (255)
        label[label >= 25] = 255 
        if self.transform is not None: image, label = self.transform(image, label)
        return image, label
"""
with open("util/dataset.py", "w") as f: f.write(dataset_code)
print("Patch Dataset applicata.")

# 3. Patch transform.py (Compatibility Fix)
trans_py = "util/transform.py"
if os.path.exists(trans_py):
    with open(trans_py, "r") as f: c = f.read()
    with open(trans_py, "w") as f: f.write(c.replace("collections.Iterable", "collections.abc.Iterable"))
    print("Patch Transform applicata.")

In [ ]:
import glob
import random


GENERATED_IMGS_DIR = "/kaggle/input/datasets/antoniorosano/translate-img/ego_cut_kaggle/test_latest/images/fake_B" 

REAL_MASKS_DIR = "/kaggle/input/datasets/antoniorosano/ego-ch/EGO-CH-OBJ-SEG/real/train/labels"


LIST_DIR = "lists/ego_ch_finetune"
os.makedirs(LIST_DIR, exist_ok=True)

print(f"Cerco immagini in: {GENERATED_IMGS_DIR}")
gen_images = glob.glob(os.path.join(GENERATED_IMGS_DIR, "*"))
pairs = []

print(f"   Trovati {len(gen_images)} file.")

for img_path in gen_images:
    basename = os.path.basename(img_path)
    clean_name = basename.replace("_fake_B", "").replace("_real_A", "").replace(".png", "").replace(".jpg", "")
    name_no_ext = os.path.splitext(clean_name)[0]
    
    mask_path = os.path.join(REAL_MASKS_DIR, name_no_ext + ".png")
    
    if not os.path.exists(mask_path):
         mask_path = os.path.join(REAL_MASKS_DIR, clean_name + ".png")

    if os.path.exists(mask_path):
        pairs.append(f"{img_path} {mask_path}")

random.seed(42)
random.shuffle(pairs)
split = int(len(pairs) * 0.9)
train_list = pairs[:split]
val_list = pairs[split:]

with open(f"{LIST_DIR}/train.txt", "w") as f: f.write("\n".join(train_list))
with open(f"{LIST_DIR}/val.txt", "w") as f: f.write("\n".join(val_list))

print(f"Liste create: {len(train_list)} train, {len(val_list)} val")

In [ ]:
PRETRAINED_PATH = "/kaggle/input/datasets/antoniorosano/psp-weights/PSPNet_logs/train_epoch_30.pth"


config_content = f"""
DATA:
  data_root: /
  train_list: /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/{LIST_DIR}/train.txt
  val_list: /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/{LIST_DIR}/val.txt
  classes: 25

TRAIN:
  arch: psp
  layers: 50
  sync_bn: False
  train_h: 473
  train_w: 473
  scale_min: 0.5
  scale_max: 2.0
  rotate_min: -10
  rotate_max: 10
  zoom_factor: 8
  ignore_label: 255
  aux_weight: 0.4
  train_gpu: [0]
  workers: 2
  batch_size: 4
  batch_size_val: 4
  
  # --- SETTAGGI FINE TUNING ---
  base_lr: 0.0005     # Molto basso per raffinare
  epochs: 50          # +20 epoche
  start_epoch: 30     # Riprendiamo da qui
  resume: {PRETRAINED_PATH}
  # --------------------------
  
  power: 0.9
  momentum: 0.9
  weight_decay: 0.0001
  manual_seed: 0
  print_freq: 10
  save_freq: 5
  save_path: exp/ego_ch_finetune/pspnet50/model
  weight: 
  evaluate: True

Distributed:
  dist_url: tcp://127.0.0.1:6789
  dist_backend: 'nccl'
  multiprocessing_distributed: False
  world_size: 1
  rank: 0
  use_apex: False
  opt_level: 'O0'
  keep_batchnorm_fp32: None
  loss_scale: None
"""

os.makedirs("config/ego_ch_finetune", exist_ok=True)
with open("config/ego_ch_finetune/pspnet50.yaml", "w") as f:
    f.write(config_content)
    
print("Configurazione salvata.")

In [ ]:
%cd /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet

import sys
import os
os.environ['PYTHONPATH'] = os.getcwd()

!pip install tensorboardX
!python tool/train.py --config config/ego_ch_finetune/pspnet50.yaml